In [ ]:
import torch
from matplotlib import pyplot as plt
import numpy as np
import cv2
import os
import uuid
import time
from sklearn.utils import shuffle


In [ ]:

IMAGES_PATH = os.path.join('images')
labels = ['drowsy', 'awake']
number_imgs = 25

In [ ]:
cap = cv2.VideoCapture(0)

for label in labels:
    print('Collecting images for {}'.format(label))
    time.sleep(5)
    
    
    for img_num in range(number_imgs):
        print('Collecting images for {}, image number {}'.format(label, img_num))
        
    
        ret, frame = cap.read()
        
    
        imgname = os.path.join(IMAGES_PATH, label+'.'+str(uuid.uuid1())+'.jpg')
        
        
        cv2.imwrite(imgname, frame)
        
        
        cv2.imshow('Image Collection', frame)
        
    
        time.sleep(2)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break
cap.release()
cv2.destroyAllWindows()

In [ ]:
def load_and_preprocess_images(image_path, label):
    images = []
    labels = []
    for img_name in os.listdir(image_path):
        img = cv2.imread(os.path.join(image_path, img_name))
        img = cv2.resize(img, (224, 224))  
        images.append(img)
        labels.append(label)
    return images, labels

closed_eye_images, closed_eye_labels = load_and_preprocess_images(CLOSED_EYES_PATH, 0)
open_eye_images, open_eye_labels = load_and_preprocess_images(OPEN_EYES_PATH, 1)

images = closed_eye_images + open_eye_images
labels = closed_eye_labels + open_eye_labels


images = np.array(images)
labels = np.array(labels)


from sklearn.utils import shuffle
images, labels = shuffle(images, labels, random_state=42)


In [ ]:
from sklearn.model_selection import train_test_split
import shutil
import os

def split_data(image_paths, labels, train_size=0.8):
    train_paths, val_paths, train_labels, val_labels = train_test_split(
        image_paths, labels, train_size=train_size, stratify=labels, random_state=42
    )
    return train_paths, val_paths, train_labels, val_labels

def move_files(image_paths, labels, dest_folder):
    for image_path, label in zip(image_paths, labels):
        
        label_file = os.path.splitext(os.path.basename(image_path))[0] + '.txt'
        label_path = os.path.join(LABELS_PATH, label_file)

        
        image_dest_folder = os.path.join(dest_folder, 'images', label)
        label_dest_folder = os.path.join(dest_folder, 'labels', label)
        
        os.makedirs(image_dest_folder, exist_ok=True)
        os.makedirs(label_dest_folder, exist_ok=True)

        
        shutil.copy(image_path, os.path.join(image_dest_folder, os.path.basename(image_path)))
        shutil.copy(label_path, os.path.join(label_dest_folder, label_file))


CLOSED_EYES_PATH = os.path.join('images', 'drowsy')
OPEN_EYES_PATH = os.path.join('images', 'awake')
LABELS_PATH = os.path.join('images', 'labels')  


image_paths = [os.path.join(CLOSED_EYES_PATH, f) for f in os.listdir(CLOSED_EYES_PATH)] + \
              [os.path.join(OPEN_EYES_PATH, f) for f in os.listdir(OPEN_EYES_PATH)]
labels = ['closed' for _ in range(len(os.listdir(CLOSED_EYES_PATH)))] + \
         ['open' for _ in range(len(os.listdir(OPEN_EYES_PATH)))]


train_paths, val_paths, train_labels, val_labels = split_data(image_paths, labels)


move_files(train_paths, train_labels, 'dataset/train')
move_files(val_paths, val_labels, 'dataset/val')


In [ ]:
!cd yolov5 && python train.py --img 320 --batch 10 --epochs 200 --data dataset.yaml --weights yolov5s.pt --workers 2


In [ ]:
from playsound import playsound

def play_alert_sound():
    playsound('alert.wav')


In [ ]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path=r'yolov5\runs\train\exp\weights\best.pt')

cap = cv2.VideoCapture(0)

drowsy_start_time = None  
DROWSY_THRESHOLD = 8  

alert_sound_path = 'alert.wav'  


drowsy_start_time = None

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture image")
        break

    results = model(frame)
    results.print()

    drowsy_detected = False
    for *box, conf, cls in results.xyxy[0]:
        if int(cls) == 17:  
            drowsy_detected = True
            break

    if drowsy_detected:
        if drowsy_start_time is None:
            drowsy_start_time = time.time()
        elif time.time() - drowsy_start_time > DROWSY_THRESHOLD:
            print("Playing alert sound now!")  
            play_alert_sound()  
    else:
        drowsy_start_time = None  


    rendered_frame = np.squeeze(results.render())
    cv2.imshow('Webcam - YOLOv5', rendered_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
